# Feature Extraction

This module intend to find the most important features for a given classification task. The main idea is to train a neural net and verify the acuracy of the predictions when ommiting a set of features.

In [1]:
from keras.models import Sequential
from keras.layers import Dense
import pandas as pd
import numpy as np
import math
import time
import itertools

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/pandas/core/computation/__init__.py:18: UserWarning: The installed version of numexpr 2.4.3 is not supported in pandas and will be not be used
The minimum supported version is 2.4.6

  ver=ver, min_ver=_MIN_NUMEXPR_VERSION), UserWarning)


In [2]:
def sigmoid(x):
    
    s=1/(1+math.exp(-x))
    return s
    

In [5]:
# fix random seed for reproducibility
np.random.seed(20)
# load pima indians dataset
dataset = np.loadtxt("./Final_IC/pima-indians-diabetes.csv", delimiter=",")
# split into input (X) and output (Y) variables
X = dataset[0:538,0:8]
Y = dataset[0:538,8]
X_test = dataset[539:768,0:8]
Y_test = dataset[539:768,8]
# create model
model = Sequential()
model.add(Dense(64,input_dim=8, activation='sigmoid'))
#model.add(Dense(64,input_dim=8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, Y, epochs=200, batch_size=5, verbose=False)
# evaluate the model
scores = model.evaluate(X_test, Y_test)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


229/229 [==============================] - 0s 56us/step

acc: 70.31%


In [3]:
def relu(x):
    s = max(0,x)
    return s

In [6]:
#(model.get_weights()[3] - model.get_weights()[3])
features_correlated=[]
taxa_de_acerto=[]
for K in range(8):
    saida=[]
    W1= model.get_weights()[0].T
    for nr_index in range(len(W1)):
        for wn_index in range(len(W1[nr_index])):
            if wn_index!=K :
                W1[nr_index][wn_index]=0
    for j in range(len(X_test)):
        N1=[]
        for i in range(64):
            N1.append(sigmoid((sum((np.multiply(W1,X_test[j])).T))[i]+model.get_weights()[1][i]))
        saida.append(round(sigmoid((sum(np.multiply(N1,model.get_weights()[2].T)[0])+model.get_weights()[3]))))
    predicted=model.predict(X_test)
    counter_acerto=0
    counter_acerto1=0
    counter_erros0=0
    counter_erros1=0
    counter_de_0=0
    counter_de_1=0
    for i in range(len(predicted)):
        if round(predicted[i][0])==0:
            if saida[i]==round(predicted[i][0]):
                counter_acerto+=1
            else:
                counter_erros0+=1
            counter_de_0+=1
        if round(predicted[i][0])==1:
            if saida[i]==round(predicted[i][0]):
                counter_acerto1+=1
            else:
                counter_erros1+=1
            counter_de_1+=1    
    taxa_acerto = counter_acerto/counter_de_0
    taxa_erro = counter_erros0/counter_de_0
    taxa_acerto1 = counter_acerto1/counter_de_1
    taxa_erro1 = counter_erros1/counter_de_1
    taxa_final = (((taxa_acerto+taxa_acerto1)/2) )
    #print (taxa_final)
    if taxa_final >0:
        features_correlated.append(K)
        taxa_de_acerto.append(taxa_final)
    

In [ ]:
#(model.get_weights()[3] - model.get_weights()[3])
features_correlated_comb=[]
taxa_acerto_comb = []
times=[]
for k in range(8):
    result_test = itertools.combinations(range(8), k)
    for comb in result_test:
        starttime = time.time()
        saida=[]
        W1= model.get_weights()[0].T
        for nr_index in range(len(W1)):
            for wn_index in range(len(W1[nr_index])):
                if wn_index not in comb :
                    W1[nr_index][wn_index]=0
        for j in range(len(X_test)):
            N1=[]
            
            for i in range(64):
                N1.append(sigmoid((sum((np.multiply(W1,X_test[j])).T))[i]+model.get_weights()[1][i]))
            saida.append(round(sigmoid((sum(np.multiply(N1,model.get_weights()[2].T)[0])+model.get_weights()[3]))))
        predicted=model.predict(X_test)
        counter_acerto=0
        counter_acerto1=0
        counter_erros0=0
        counter_erros1=0
        counter_de_0=0
        counter_de_1=0
        for i in range(len(predicted)):
            if round(predicted[i][0])==0:
                if saida[i]==round(predicted[i][0]):
                    counter_acerto+=1
                else:
                    counter_erros0+=1
                counter_de_0+=1
            if round(predicted[i][0])==1:
                if saida[i]==round(predicted[i][0]):
                    counter_acerto1+=1
                else:
                    counter_erros1+=1
                counter_de_1+=1    
        taxa_acerto = counter_acerto/counter_de_0
        taxa_erro = counter_erros0/counter_de_0
        taxa_acerto1 = counter_acerto1/counter_de_1
        taxa_erro1 = counter_erros1/counter_de_1
        taxa_final = (((taxa_acerto+taxa_acerto1)/2) )
        #print (taxa_final)
        if taxa_final >0:
            features_correlated_comb.append(comb)
            taxa_acerto_comb.append(taxa_final)
        times.append(time.time()-starttime)

In [10]:

taxa_acerto_comb, features_correlated_comb = zip(*sorted(zip(taxa_acerto_comb, features_correlated_comb),reverse=True))
for i in range(1,8):
    for comb in range(len(features_correlated_comb)):  
        if len(features_correlated_comb[comb])==i:
            print (features_correlated_comb[comb],taxa_acerto_comb[comb])
            break

(0,) 0.6563299232736572
(0, 6) 0.6728169528681038
(0, 3, 6) 0.6854219948849105
(0, 5, 6, 7) 0.6359152356594812
(0, 1, 2, 5, 7) 0.7648428936792109
(0, 1, 2, 5, 6, 7) 0.8069967117281696
(0, 1, 2, 3, 4, 5, 7) 0.9338235294117647


## Get most important feature_value

It was not fully implemented because in the early research the time to compute the combinations was too high. A possible alternative is to use heuristics.

In [ ]:
'''result_test = itertools.combinations(range(8), 7)
countii = 0
for comb in result_test:
    print (comb)
    countii+=1
countii'''

In [ ]:
'''X_train = pd.DataFrame(dataset[:,0:8])
X_train_dummy = pd.DataFrame()
for col in X_train.columns:
    for val in (np.unique(X_train[col])):
        colname=str(col)+"_"+str(val)
        
        X_train_dummy[colname]= X_train[col]==val'''

In [ ]:
'''X = np.asarray(X_train_dummy[0:668])
X_test = np.asarray(X_train_dummy[669:768])
Y = dataset[0:668,8]
Y_test = dataset[669:768,8]
# create model
model = Sequential()
model.add(Dense(64,input_dim=len(X_train_dummy.columns), activation='sigmoid'))
#model.add(Dense(64,input_dim=8, activation='sigmoid'))
model.add(Dense(1, activation='sigmoid'))
# Compile model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
# Fit the model
model.fit(X, Y, epochs=200, batch_size=1, verbose=False)
# evaluate the model
scores = model.evaluate(X_test, Y_test)

print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
'''

In [ ]:
'''#(model.get_weights()[3] - model.get_weights()[3])
features_correlated_comb=[]
taxa_acerto_comb = []
times=[]
for k in range(1,2):
    result_test = itertools.combinations(range(len(X_train_dummy.columns)), k)
    for comb in result_test:
        starttime = time.time()
        saida=[]
        W1= model.get_weights()[0].T
        for nr_index in range(len(W1)):
            for wn_index in range(len(W1[nr_index])):
                if wn_index not in comb :
                    W1[nr_index][wn_index]=0
        
        for j in range(len(X_test)):
            N1=[]
            for i in range(64):
                N1.append(sigmoid((sum((np.multiply(W1,X_test[j])).T))[i]+model.get_weights()[1][i]))
            saida.append(round(sigmoid((sum(np.multiply(N1,model.get_weights()[2].T)[0])+model.get_weights()[3]))))
        predicted=model.predict(X_test)
        
        
        counter_acerto=0
        counter_acerto1=0
        counter_erros0=0
        counter_erros1=0
        counter_de_0=0
        counter_de_1=0
        for i in range(len(predicted)):
            if round(predicted[i][0])==0:
                if saida[i]==round(predicted[i][0]):
                    counter_acerto+=1
                else:
                    counter_erros0+=1
                counter_de_0+=1
            if round(predicted[i][0])==1:
                if saida[i]==round(predicted[i][0]):
                    counter_acerto1+=1
                else:
                    counter_erros1+=1
                counter_de_1+=1    
        taxa_acerto = counter_acerto/counter_de_0
        taxa_erro = counter_erros0/counter_de_0
        taxa_acerto1 = counter_acerto1/counter_de_1
        taxa_erro1 = counter_erros1/counter_de_1
        taxa_final = (((taxa_acerto+taxa_acerto1)/2) - ((taxa_erro+taxa_erro1)/2))
        print(comb)
        print (taxa_final)
        if taxa_final >0:
            features_correlated_comb.append(comb)
            taxa_acerto_comb.append(taxa_final)
        
        times.append(time.time()-starttime)'''

In [ ]:
'''dataset_frame = pd.DataFrame(dataset)
dataset_frame'''